# TMDB 5000 Movie Dataset으로 장르기반 추천시스템 만들기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ablearn/main/tmdb_5000_movies.csv")

In [3]:
dic_cols = ['genres', 'keywords', 'production_companies', "production_countries", "spoken_languages"]

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [5]:
data.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
data['genres'].describe()

count                              4803
unique                             1175
top       [{"id": 18, "name": "Drama"}]
freq                                370
Name: genres, dtype: object

In [7]:
len(data['genres'][4])

103

In [8]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [9]:
import json

temp = [] 
for i in json.loads(data['genres'][0]):
#     print(type(i))
    temp.append(i.get('name', 'None'))
temp
" ".join(temp)

'Action Adventure Fantasy Science Fiction'

In [10]:
def dic_unpack(data):
    for col in ['genres', 'keywords', 'production_companies', "production_countries", "spoken_languages"]:
        data[col] = [i.get('name', 'None') for i in json.loads(data[col])]
        data[col] = " ".join(data[col])
    return data

In [11]:
data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [12]:
data

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [13]:
data = data[['original_title', 'genres', 'popularity', 'runtime', 'vote_average', 'vote_count']]
data.head()

,original_title,genres,popularity,runtime,vote_average,vote_count
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",150.437577,162.0,7.2,11800
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",139.082615,169.0,6.9,4500
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",107.376788,148.0,6.3,4466
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",112.312950,165.0,7.6,9106
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",43.926995,132.0,6.1,2124


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4803 non-null   object 
 1   genres          4803 non-null   object 
 2   popularity      4803 non-null   float64
 3   runtime         4801 non-null   float64
 4   vote_average    4803 non-null   float64
 5   vote_count      4803 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 225.3+ KB


In [15]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4801 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   original_title  4801 non-null   object 
 1   genres          4801 non-null   object 
 2   popularity      4801 non-null   float64
 3   runtime         4801 non-null   float64
 4   vote_average    4801 non-null   float64
 5   vote_count      4801 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 262.6+ KB


- genres의 단어를 벡터화
- 코사인유사도를 구해 비슷한 장르의 행 검색
- 유사도가 높은 행을 기준으로 내림차순 정렬

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
cv = CountVectorizer(ngram_range=(1, 2))
genres_mat = cv.fit_transform(data['genres'])
print(genres_mat)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 78807 stored elements and shape (4801, 126)>
  Coords	Values
  (0, 66)	4
  (0, 22)	1
  (0, 93)	4
  (0, 40)	1
  (0, 12)	1
  (0, 42)	1
  (0, 14)	1
  (0, 56)	1
  (0, 34)	1
  (0, 116)	1
  (0, 58)	1
  (0, 78)	1
  (0, 23)	1
  (0, 94)	1
  (0, 41)	1
  (0, 73)	1
  (0, 13)	1
  (0, 95)	1
  (0, 43)	1
  (0, 74)	1
  (0, 15)	1
  (0, 102)	1
  (0, 57)	1
  (0, 84)	1
  (0, 35)	1
  :	:
  (4798, 46)	1
  (4798, 79)	1
  (4798, 25)	1
  (4798, 97)	1
  (4798, 47)	1
  (4798, 2)	1
  (4798, 114)	1
  (4798, 68)	1
  (4798, 3)	1
  (4798, 108)	1
  (4798, 115)	1
  (4798, 10)	1
  (4798, 120)	1
  (4798, 87)	1
  (4798, 72)	1
  (4798, 11)	1
  (4798, 111)	1
  (4798, 121)	1
  (4800, 66)	1
  (4800, 93)	1
  (4800, 38)	1
  (4800, 50)	1
  (4800, 86)	1
  (4800, 39)	1
  (4800, 99)	1


In [18]:
genres_mat.shape

(4801, 126)

### 코사인 유사도를 사용해 genres_mat의 유사도를 산출 후 genres_sim

- 비슷한 장르의 영화를 찾기위해 계산
- 코사인 유사도는 주로 문자, 문자의 유사도를 계산하는데 사용됨

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
genres_sim = cosine_similarity(genres_mat, genres_mat)
print(genres_sim.shape)

(4801, 4801)


In [21]:
genres_sim[:10]

array([[1.        , 0.91793601, 0.80599259, ..., 0.56140351, 0.        ,
        0.40050094],
       [0.91793601, 1.        , 0.82857143, ..., 0.5373284 , 0.        ,
        0.38332594],
       [0.80599259, 0.82857143, 1.        , ..., 0.5373284 , 0.        ,
        0.38332594],
       ...,
       [0.93633265, 0.80586652, 0.83365502, ..., 0.52260427, 0.        ,
        0.37282186],
       [0.80599259, 0.85714286, 0.68571429, ..., 0.5373284 , 0.        ,
        0.38332594],
       [0.91793601, 0.97142857, 0.85714286, ..., 0.5373284 , 0.        ,
        0.38332594]])

In [22]:
sorted_genre_sim = genres_sim.argsort()[:,::-1]

In [23]:
sorted_genre_sim[:1]

array([[  14,    0,   46, ..., 3970, 3991, 4799]])

In [24]:
data.iloc[0, :]

original_title                                               Avatar
genres            [{"id": 28, "name": "Action"}, {"id": 12, "nam...
popularity                                               150.437577
runtime                                                       162.0
vote_average                                                    7.2
vote_count                                                    11800
Name: 0, dtype: object

In [25]:
data.iloc[870, :]

original_title                                          Superman II
genres            [{"id": 28, "name": "Action"}, {"id": 12, "nam...
popularity                                                30.515175
runtime                                                       127.0
vote_average                                                    6.5
vote_count                                                      629
Name: 870, dtype: object

# 영화 이름과 추천 받을 개수를 입력받아 출력하는 함수

In [39]:
def find_sim_movie(data, sorted_genre_sim, title_name, top_n=10):
    title_movie = data[data['original_title'] == title_name]
    title_index = title_movie.index.values[0] # 원본 데이터 프레임에서 영화 제목에 맞는 인덱스 추출
    similar_indexes = sorted_genre_sim[title_index, 1 : (top_n+1)] # 추출한 인덱스로 코사인유사도 결과 추출
    print("similar_indexes", similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    return data.iloc[similar_indexes] # 추출한 코사인 유사도 결과로 원본 데이터에서 영화 추천

In [40]:
sorted_genre_sim[0,1:(11)]

array([   0,   46,  813,  870, 3493,   10,   61,  232, 1230, 1932])

In [41]:
data.iloc[sorted_genre_sim[0,1:(11)].reshape(-1), :]

,original_title,genres,popularity,runtime,vote_average,vote_count
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",150.437577,162.0,7.2,11800
46,X-Men: Days of Future Past,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",118.078691,131.0,7.5,6032
813,Superman,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",48.507081,143.0,6.9,1022
870,Superman II,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",30.515175,127.0,6.5,629
3494,Beastmaster 2: Through the Portal of Time,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",1.478505,107.0,4.6,17
10,Superman Returns,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",57.925623,154.0,5.4,1400
61,Jupiter Ascending,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",85.369080,124.0,5.2,2768
232,The Wolverine,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",15.953444,126.0,6.3,4053
1230,The Shadow,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",8.178203,108.0,5.4,138
1932,Sheena,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",4.020194,117.0,5.0,22


In [43]:
find_sim_movie(data, sorted_genre_sim, 'Avatar', 20)

similar_indexes [   0   46  813  870 3493   10   61  232 1230 1932 1296  238  618 1390
 1652 3207  322   72 1191   47]


,original_title,genres,popularity,runtime,vote_average,vote_count
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",150.437577,162.0,7.2,11800
46,X-Men: Days of Future Past,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",118.078691,131.0,7.5,6032
813,Superman,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",48.507081,143.0,6.9,1022
870,Superman II,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",30.515175,127.0,6.5,629
3494,Beastmaster 2: Through the Portal of Time,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",1.478505,107.0,4.6,17
10,Superman Returns,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",57.925623,154.0,5.4,1400
61,Jupiter Ascending,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",85.369080,124.0,5.2,2768
232,The Wolverine,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...",15.953444,126.0,6.3,4053
1230,The Shadow,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",8.178203,108.0,5.4,138
1932,Sheena,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",4.020194,117.0,5.0,22


In [44]:
title = input("영화 제목을 정확히 입력하세요")
find_sim_movie(data, sorted_genre_sim, title)

영화 제목을 정확히 입력하세요 Superman III


similar_indexes [1296 1932 1191  238  859 2069   14 3493  342 1936]


,original_title,genres,popularity,runtime,vote_average,vote_count
1296,Superman III,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 28, ""nam...",22.164202,125.0,5.3,490
1932,Sheena,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",4.020194,117.0,5.0,22
1191,Small Soldiers,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 12, ""nam...",23.088571,110.0,6.2,511
238,Teenage Mutant Ninja Turtles,"[{""id"": 878, ""name"": ""Science Fiction""}, {""id""...",143.350376,101.0,5.8,2636
859,Thunderbirds,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",9.278750,95.0,4.2,91
2069,Teenage Mutant Ninja Turtles III,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",10.207065,96.0,5.1,184
14,Man of Steel,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",99.398009,143.0,6.5,6359
3494,Beastmaster 2: Through the Portal of Time,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",1.478505,107.0,4.6,17
342,Men in Black,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",104.121555,98.0,6.9,4412
1936,Tank Girl,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",8.186936,98.0,5.5,138
